In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train = pd.read_csv('Data/train.csv',index_col = 0)

In [3]:
test = pd.read_csv('Data/test.csv',index_col = 0)

In [4]:
train.drop(columns = ['Stoich. Formula', 'Point Group','split','rad_mean','rad_var','SG_vec','SG_freq'],inplace=True)

In [5]:
test.drop(columns = ['Stoich. Formula', 'Point Group','split','rad_mean','rad_var','SG_vec','SG_freq'],inplace=True)

In [12]:
y_0 = []
y_TI = []
y_TSM = []
for i in range(len(train)):
    if train['Top_Class'].iloc[i]==0:
        y_0.append(0)
        y_TI.append(0)
        y_TSM.append(0)
    elif train['Top_Class'].iloc[i]==1 or train['Top_Class'].iloc[i]==2:
        y_0.append(1)
        y_TI.append(1)
        y_TSM.append(0)  
    else:
        y_0.append(1)
        y_TI.append(0)
        y_TSM.append(1)  

In [18]:
y_0 = []
y_TI = []
y_TSM = []
for i in range(len(test)):
    if test['Top_Class'].iloc[i]==0:
        y_0.append(0)
        y_TI.append(0)
        y_TSM.append(0)
    elif test['Top_Class'].iloc[i]==1 or test['Top_Class'].iloc[i]==2:
        y_0.append(1)
        y_TI.append(1)
        y_TSM.append(0)  
    else:
        y_0.append(1)
        y_TI.append(0)
        y_TSM.append(1)  

In [14]:
train['y_triv'] = y_0
train['y_TI'] = y_TI
train['y_TSM'] = y_TSM

In [19]:
test['y_triv'] = y_0
test['y_TI'] = y_TI
test['y_TSM'] = y_TSM

In [23]:
def fit_GBT(X,y,X_test,y_test):

    model = XGBClassifier(use_label_encoder=False, eta=0.3)
    param_grid = {'max_depth':[10,30,100],'min_child_weight':[5,6],'colsample_bytree' : [0.5,0.6,0.7,0.8,0.9,1],'colsample_bynode' : [0.5,0.6,0.7,0.8,0.9,1],'lambda' : [0.01,0.03,0.1,0.3,1,10,30]}

    gs = RandomizedSearchCV(model,param_distributions = param_grid, n_jobs = -1, n_iter = 50)
    results = gs.fit(X,y)
    print('Results of randomized search ...')

    #display(pd.DataFrame(results.cv_results_).sort_values('rank_test_score'))
    print('='*50)
    print('Best Parameters:')
    display(results.best_params_)
    clf = XGBClassifier(**results.best_params_)
    clf.fit(X,y)
    y_pred_train = clf.predict(X)
    y_pred_test = clf.predict(X_test)
    print('Precision, Recall and F1 score for train data')
    display(metrics.precision_recall_fscore_support(y,y_pred_train))
    print('Precision, Recall and F1 score for test data')
    display(metrics.precision_recall_fscore_support(y_test,y_pred_test))
    print('Accuracy on train data')
    display(metrics.accuracy_score(y,y_pred_train))
    print('Accuracy on test data')
    display(metrics.accuracy_score(y_test,y_pred_test))
    #acc_scores.append(metrics.accuracy_score(y_test,y_pred_test))
    print('='*50)
#    print('Plotting feature importances .. ')
#     fig, ax = plt.subplots(figsize=(15, 15))
#     plot_importance(clf, ax=ax)

## Top or not

In [21]:
fit_GBT(train.drop(columns = ['Top_Class','y_TI','y_TSM','y_triv']),train['y_triv'],test.drop(columns = ['Top_Class','y_TI','y_TSM','y_triv']),test['y_triv'])

[15:39:48] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Results of randomized search ...
Best Parameters:


{'min_child_weight': 5,
 'max_depth': 10,
 'lambda': 0.3,
 'colsample_bytree': 0.8,
 'colsample_bynode': 0.9}

[15:39:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/chaithanya/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Precision, Recall and F1 score for train data


...

0.9642449600608597

NameError: name 'acc_scores' is not defined

[15:29:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:29:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:29:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:30:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[15:29:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:29:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:29:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:30:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[15:29:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:29:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:29:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:30:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

## TI vs all

In [22]:
fit_GBT(train.drop(columns = ['Top_Class','y_TI','y_TSM','y_triv']),train['y_TI'],test.drop(columns = ['Top_Class','y_TI','y_TSM','y_triv']),test['y_TI'])

[16:17:19] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Results of randomized search ...
Best Parameters:


{'min_child_weight': 5,
 'max_depth': 10,
 'lambda': 0.01,
 'colsample_bytree': 0.5,
 'colsample_bynode': 0.5}

[16:17:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/chaithanya/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Precision, Recall and F1 score for train data


...

Accuracy on train data


0.9931519878257561

Accuracy on test data


0.900342335488779

NameError: name 'acc_scores' is not defined

In [24]:
fit_GBT(train.drop(columns = ['Top_Class','y_TI','y_TSM','y_triv']),train['y_TSM'],test.drop(columns = ['Top_Class','y_TI','y_TSM','y_triv']),test['y_TSM'])

[16:29:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Results of randomized search ...
Best Parameters:


{'min_child_weight': 5,
 'max_depth': 10,
 'lambda': 0.03,
 'colsample_bytree': 0.6,
 'colsample_bynode': 0.9}

[16:29:29] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/chaithanya/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Precision, Recall and F1 score for train data


...

[16:07:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:08:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[16:07:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:08:06] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[16:07:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:08:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[16:07:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:08:06] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

[16:07:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:07:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:08:08] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

## Random Forests

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
def fit_RF(X,y,X_test,y_test):

    model = RandomForestClassifier(class_weight='balanced',random_state=0)
    param_grid = {'n_estimators':[10,25,50,100,200],'criterion':['gini', 'entropy'],'max_depth':[10,30,50,100,200]}

    gs = RandomizedSearchCV(model,param_distributions = param_grid, n_jobs = -1, n_iter = 50)
    results = gs.fit(X,y)
    print('Results of randomized search ...')

    #display(pd.DataFrame(results.cv_results_).sort_values('rank_test_score'))
    print('='*50)
    print('Best Parameters:')
    display(results.best_params_)
    clf = RandomForestClassifier(**results.best_params_)
    clf.fit(X,y)
    y_pred_train = clf.predict(X)
    y_pred_test = clf.predict(X_test)
    print('Precision, Recall and F1 score for train data')
    display(metrics.precision_recall_fscore_support(y,y_pred_train))
    print('Precision, Recall and F1 score for test data')
    display(metrics.precision_recall_fscore_support(y_test,y_pred_test))
    print('Accuracy on train data')
    display(metrics.accuracy_score(y,y_pred_train))
    print('Accuracy on test data')
    display(metrics.accuracy_score(y_test,y_pred_test))
    #acc_scores.append(metrics.accuracy_score(y_test,y_pred_test))
    print('='*50)
#    print('Plotting feature importances .. ')
#     fig, ax = plt.subplots(figsize=(15, 15))
#     plot_importance(clf, ax=ax)

In [10]:
fit_RF(train.drop(columns = ['Top_Class']),train['Top_Class'],test.drop(columns = ['Top_Class']),test['Top_Class'])

Results of randomized search ...
Best Parameters:


{'n_estimators': 200, 'max_depth': 50, 'criterion': 'gini'}

Precision, Recall and F1 score for train data


(array([0.99978289, 0.99032258, 0.99353647, 0.98556999, 0.99788248]),
 array([0.99978289, 0.98208573, 0.99445471, 0.99201162, 0.99946978]),
 array([0.99978289, 0.98618696, 0.99399538, 0.98878031, 0.9986755 ]),
 array([4606, 1563, 1082, 1377, 1886]))

Precision, Recall and F1 score for test data


(array([0.85780169, 0.84971098, 0.69767442, 0.76923077, 0.77125506]),
 array([0.96875   , 0.75191816, 0.55350554, 0.61046512, 0.8089172 ]),
 array([0.90990624, 0.79782904, 0.61728395, 0.68071313, 0.78963731]),
 array([1152,  391,  271,  344,  471]))

Accuracy on train data


0.995529769830702

Accuracy on test data


0.8181818181818182

## KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [23]:
def fit_knn(X,y,X_test,y_test):

    model = KNeighborsClassifier()
    param_grid = {'weights':['uniform', 'distance'],'n_neighbors':[10,15,20,30,50,75,100,150]}

    gs = RandomizedSearchCV(model,param_distributions = param_grid, n_jobs = -1, n_iter = 50)
    results = gs.fit(X,y)
    print('Results of randomized search ...')

    #display(pd.DataFrame(results.cv_results_).sort_values('rank_test_score'))
    print('='*50)
    print('Best Parameters:')
    display(results.best_params_)
    clf = KNeighborsClassifier(**results.best_params_)
    clf.fit(X,y)
    y_pred_train = clf.predict(X)
    y_pred_test = clf.predict(X_test)
    print('Precision, Recall and F1 score for train data')
    display(metrics.precision_recall_fscore_support(y,y_pred_train))
    print('Precision, Recall and F1 score for test data')
    display(metrics.precision_recall_fscore_support(y_test,y_pred_test))
    print('Accuracy on train data')
    display(metrics.accuracy_score(y,y_pred_train))
    print('Accuracy on test data')
    display(metrics.accuracy_score(y_test,y_pred_test))
    #acc_scores.append(metrics.accuracy_score(y_test,y_pred_test))
    print('='*50)
#    print('Plotting feature importances .. ')
#     fig, ax = plt.subplots(figsize=(15, 15))
#     plot_importance(clf, ax=ax)

In [24]:
fit_knn(train.drop(columns = ['Top_Class']),train['Top_Class'],test.drop(columns = ['Top_Class']),test['Top_Class'])

/Users/chaithanya/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 16 is smaller than n_iter=50. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Results of randomized search ...
Best Parameters:


{'weights': 'distance', 'n_neighbors': 100}

Precision, Recall and F1 score for train data


(array([0.99978142, 0.94785276, 0.99626168, 0.9890431 , 0.99893048]),
 array([0.99305254, 0.98848369, 0.98521257, 0.98329702, 0.99045599]),
 array([0.99640562, 0.96774194, 0.99070632, 0.98616169, 0.99467519]),
 array([4606, 1563, 1082, 1377, 1886]))

...

(array([0.72571429, 0.63360882, 0.585     , 0.73275862, 0.60368664]),
 array([0.88194444, 0.58823529, 0.43173432, 0.49418605, 0.55626327]),
 array([0.79623824, 0.61007958, 0.49681529, 0.59027778, 0.57900552]),
 array([1152,  391,  271,  344,  471]))

Accuracy on train data


0.989823093018832

Accuracy on test data


0.6827691137314569

## SVM

In [25]:
from sklearn.svm import SVC

In [29]:
def fit_svm(X,y,X_test,y_test):

    model = SVC()
    param_grid = {'C':[0.001,0.01,0.1,1,10,100],'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'degree':[2,3,5,7],'gamma':['scale','auto']}

    gs = RandomizedSearchCV(model,param_distributions = param_grid, n_jobs = -1, n_iter = 50)
    results = gs.fit(X,y)
    print('Results of randomized search ...')

    #display(pd.DataFrame(results.cv_results_).sort_values('rank_test_score'))
    print('='*50)
    print('Best Parameters:')
    display(results.best_params_)
    clf = SVC(**results.best_params_)
    clf.fit(X,y)
    y_pred_train = clf.predict(X)
    y_pred_test = clf.predict(X_test)
    print('Precision, Recall and F1 score for train data')
    display(metrics.precision_recall_fscore_support(y,y_pred_train))
    print('Precision, Recall and F1 score for test data')
    display(metrics.precision_recall_fscore_support(y_test,y_pred_test))
    print('Accuracy on train data')
    display(metrics.accuracy_score(y,y_pred_train))
    print('Accuracy on test data')
    display(metrics.accuracy_score(y_test,y_pred_test))
    #acc_scores.append(metrics.accuracy_score(y_test,y_pred_test))
    print('='*50)
#    print('Plotting feature importances .. ')
#     fig, ax = plt.subplots(figsize=(15, 15))
#     plot_importance(clf, ax=ax)

In [30]:
fit_svm(train.drop(columns = ['Top_Class']),train['Top_Class'],test.drop(columns = ['Top_Class']),test['Top_Class'])

KeyboardInterrupt: 